In [86]:
import sys
sys.path.append('../')
from utilities.task_service import get_authentication_token, get_task, send_answer
from auth import apikey
import requests
import json

from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
from langchain_openai import OpenAIEmbeddings
from uuid import uuid4
from qdrant_client import QdrantClient
from langchain_community.document_loaders import TextLoader
from langchain.docstore.document import Document

from dotenv import load_dotenv
load_dotenv(override=True)

task_name = "search"

In [89]:
# pobranie linków z urla
url = f"https://unknow.news/archiwum_aidevs.json"

try:
    response_links = requests.post(url)
    response_links.raise_for_status()  # Sprawdzenie, czy odpowiedź jest sukcesem (status kod 2xx)
    links = response_links.json()
except requests.exceptions.RequestException as e:
    print(f"Request error: {e}")

with open("links.json", "w", encoding='utf8') as file:
    json.dump(links, file, indent=1)

In [92]:
# połączenie z Qdrant
#MEMORY_PATH = "links.json"
COLLECTION_NAME = "search_task"

qdrant = QdrantClient()
embeddings = OpenAIEmbeddings()
result = qdrant.get_collections()
indexed = next((collection for collection in result.collections if collection.name == COLLECTION_NAME), None)
print(result)

# stworzenie kolekcji jeśli nie istnieje
if not indexed:
    qdrant.recreate_collection(
        collection_name=COLLECTION_NAME,
        vectors_config={"size": 1536, "distance": "Cosine", "on_disk": True},
    )

collection_info = qdrant.get_collection(collection_name=COLLECTION_NAME)

collections=[CollectionDescription(name='ai_devs')]


In [93]:
# jeśli nie ma dokumentów w kolekcji
if not collection_info.points_count:
    # dodanie id do każdego linku
    for link in links:
        link["uuid"] = str(uuid4())
        link['source'] = COLLECTION_NAME

    # generowanie embeddingów
    points = []
    for link in links:
        embedding = embeddings.embed_documents([link['info']])[0]
        points.append(
            {
                "id": link['uuid'],
                "payload": link,
                "vector": embedding,
            }
        )

    # index
    qdrant.upsert(
        collection_name=COLLECTION_NAME,
        wait=True,
        points=points,
    )

# collection_info = qdrant.get_collection(collection_name=COLLECTION_NAME)
# json.loads(collection_info.json())

In [108]:
# autoryzacja

token = get_authentication_token(apikey, task_name)
if token:
    print("Otrzymano token.")
else:
    print("Nie udało się uzyskać tokenu.")

Otrzymano token.


In [109]:
# pobranie danych do zadania

response = get_task(token)
if response:
    print("Odpowiedź z serwera:", response)
else:
    print("Wystąpił błąd podczas wysyłania zapytania.")

Odpowiedź z serwera: {'code': 0, 'msg': 'Index all data from provided URL into vecto store and provide answer to my question - https://unknow.news/archiwum_aidevs.json', 'question': 'Co różni pseudonimizację od anonimizowania danych?'}


In [110]:
# przeszukiwanie dokumentów w wybranej kolekcji powiązanych z pytaniem
query = response['question']
query_embedding = embeddings.embed_query(query)

search_result = qdrant.search(
    collection_name=COLLECTION_NAME,
    query_vector=query_embedding,
    limit=1,
    query_filter={"must": [{"key": "source", "match": {"value": COLLECTION_NAME}}]},
)

for result in search_result:
    print("ID: ", result.id)
    print("Score: ", result.score)
    print(json.dumps(result.payload,indent=4, ensure_ascii=False))
    answer = result.payload['url']

ID:  0cc7391b-ba02-41f4-adc4-0d48f2b696b8
Score:  0.8988228
{
    "date": "2023-10-06",
    "info": "INFO: Czy patrząc na dane, można jednoznacznie stwierdzić, że są anonimowe? Czy np. haszowanie numerów telefonu to skuteczna pseudonimizacja? Adwokatka i kryptolog podejmują próbę pogodzenia prawnych i matematycznych definicji pojęć \"anonimizacja\" i \"pseudonimizacja\".",
    "source": "search_task",
    "title": "Czym się różni pseudonimizacja od anonimizacji? (film, 46 minut)",
    "url": "https://www.internet-czas-dzialac.pl/pseudonimizacja-a-anonimizacja/",
    "uuid": "0cc7391b-ba02-41f4-adc4-0d48f2b696b8"
}


In [111]:
# wysłanie odpowiedzi

response = send_answer(token, answer)

if response:
    print("Odpowiedź z serwera:", response)
else:
    print("Wystąpił błąd podczas wysyłania odpowiedzi.")

Odpowiedź z serwera: {'code': 0, 'msg': 'OK', 'note': 'CORRECT'}
